using example in https://huggingface.co/blog/how-to-train
https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=QDNgPls7_l13


work in progress

In [16]:
import os
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

In [17]:
import transformers
import tokenizers


notebook_path = os.path.abspath("RoBERTa.ipynb")

In [18]:
tokenizer = ByteLevelBPETokenizer(
    "./../tokenize/vocab.json",
    "./../tokenize/merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)
# or use the RobertaTokenizer from `transformers` directly.

examples = []

src_file = os.path.realpath("../tokenize/pre-tokenized/modified_cnn.txt")

with open(src_file, 'r', encoding="ISO-8859-1") as f:
        try:
            rawContent = f.read() 
        except Exception as e:
            print('During file reading, this error occurred:', e)


# tokenizer.encode(rawContent)
tokenizer.encode(rawContent).tokens

tokenizer.save("byte-level-BPE.tokenizer.json")

In [19]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)


In [20]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

model.num_parameters()

84095008

In [21]:
from transformers import LineByLineTextDataset
from transformers import PreTrainedTokenizerFast


# Debug through https://github.com/huggingface/transformers/issues/7234
tokenizer = PreTrainedTokenizerFast(tokenizer_file="byte-level-BPE.tokenizer.json")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Just training on one point for debugging
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="../tokenize/pre-tokenized/modified_cnn.txt",
    block_size=128,
)


from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, mlm_probability=0.15
)

TypeError: __init__() missing 1 required positional argument: 'tokenizer'

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir= "./model_one",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
trainer.train()
trainer.save_model("./model_one")

In [ ]:
# to check if everything is working later?
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="model_one",
    tokenizer=tokenizer,
)


result = fill_mask("La suno <mask>.")
print(result)